In [1]:
# importing all the important libraries
import re
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [2]:
sw = stopwords.words('english')

In [3]:
# read the positvie text data
pos_rev = pd.read_csv('pos.txt' , sep='\n' , header = None , encoding = 'latin-1')

# adding a tareget column
pos_rev['mood'] = 1.0
pos_rev = pos_rev.rename(columns = {0:'review'})
pos_rev

,review,mood
0,the rock is destined to be the 21st century's ...,1.0
1,"the gorgeously elaborate continuation of "" the...",1.0
2,effective but too-tepid biopic,1.0
3,if you sometimes like to go to the movies to h...,1.0
4,"emerges as something rare , an issue movie tha...",1.0
...,...,...
5326,both exuberantly romantic and serenely melanch...,1.0
5327,mazel tov to a film about a family's joyous li...,1.0
5328,standing in the shadows of motown is the best ...,1.0
5329,it's nice to see piscopo again after all these...,1.0


In [4]:
pos_rev

,review,mood
0,the rock is destined to be the 21st century's ...,1.0
1,"the gorgeously elaborate continuation of "" the...",1.0
2,effective but too-tepid biopic,1.0
3,if you sometimes like to go to the movies to h...,1.0
4,"emerges as something rare , an issue movie tha...",1.0
...,...,...
5326,both exuberantly romantic and serenely melanch...,1.0
5327,mazel tov to a film about a family's joyous li...,1.0
5328,standing in the shadows of motown is the best ...,1.0
5329,it's nice to see piscopo again after all these...,1.0


In [5]:
# read the negative text data
neg_rev = pd.read_csv('negative.txt' , sep='\n' , header = None , encoding = 'latin-1')

# adding a tareget column
neg_rev['mood'] = 0.0
neg_rev = neg_rev.rename(columns = {0:'review'})
neg_rev

,review,mood
0,"simplistic , silly and tedious.",0.0
1,"it's so laddish and juvenile , only teenage bo...",0.0
2,exploitative and largely devoid of the depth o...,0.0
3,[garbus] discards the potential for pathologic...,0.0
4,a visually flashy but narratively opaque and e...,0.0
...,...,...
5326,a terrible movie that some people will neverth...,0.0
5327,there are many definitions of 'time waster' bu...,0.0
5328,"as it stands , crocodile hunter has the hurrie...",0.0
5329,the thing looks like a made-for-home-video qui...,0.0


In [6]:
# cleaning the data

# 1. lower
# 2. remove spaces
# 3. punctuation
# 4. stopwords
# 5. lemmatize

In [7]:
import string
punct=string.punctuation
print(punct)
q=dict.fromkeys(punct)
print(str.maketrans(q))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

ps=PorterStemmer()
wordnet= WordNetLemmatizer()

In [9]:
pos_rev.loc[: , 'review'] = pos_rev.loc[: , 'review'].apply(lambda x : x.lower())
pos_rev.loc[: , 'review'] = pos_rev.loc[: , 'review'].apply(lambda x : re.sub(r"@\S+" , "" , x)) #@... is replace with '' or it is removed
pos_rev.loc[: , 'review'] = pos_rev.loc[: , 'review'].apply\
(lambda x : x.translate(str.maketrans(dict.fromkeys(string.punctuation))))
pos_rev.loc[: , 'review'] = pos_rev.loc[: , 'review'].apply\
(lambda x : " ".join([word for word in x.split() if word not in (sw)]))
pos_rev.loc[: , 'review'] =pos_rev.loc[: , 'review'].apply\
(lambda x :" ".join([wordnet.lemmatize(word) for word in x.split()]))

In [10]:
neg_rev.loc[: , 'review'] = neg_rev.loc[: , 'review'].apply(lambda x : x.lower())
neg_rev.loc[: , 'review'] = neg_rev.loc[: , 'review'].apply(lambda x : re.sub(r"@\S+" , "" , x))
neg_rev.loc[: , 'review'] = neg_rev.loc[: , 'review'].apply\
(lambda x : x.translate(str.maketrans(dict.fromkeys(string.punctuation))))
neg_rev.loc[: , 'review'] = neg_rev.loc[: , 'review'].apply\
(lambda x : " ".join([word for word in x.split() if word not in (sw)]))
neg_rev.loc[: , 'review'] =neg_rev.loc[: , 'review'].apply\
(lambda x :" ".join([wordnet.lemmatize(word) for word in x.split()]))

In [11]:
# concatunatiing the pos and negative data
com_rev = pd.concat([pos_rev , neg_rev] , axis = 0).reset_index()
com_rev

,index,review,mood
0,0,rock destined 21st century new conan he going ...,1.0
1,1,gorgeously elaborate continuation lord ring tr...,1.0
2,2,effective tootepid biopic,1.0
3,3,sometimes like go movie fun wasabi good place ...,1.0
4,4,emerges something rare issue movie thats hones...,1.0
...,...,...,...
10657,5326,terrible movie people nevertheless find moving,0.0
10658,5327,many definition time waster movie must surely one,0.0
10659,5328,stand crocodile hunter hurried badly cobbled l...,0.0
10660,5329,thing look like madeforhomevideo quickie,0.0


In [12]:
# train_test_split
X_train , X_test, y_train, y_test = train_test_split\
(com_rev['review'].values , com_rev['mood'].values , test_size = 0.2 , random_state = 101)

In [13]:
train_data = pd.DataFrame({'review':X_train , 'mood':y_train})
test_data = pd.DataFrame({'review':X_test , 'mood':y_test})

In [14]:
train_data

,review,mood
0,put washington honest working man john q archi...,0.0
1,poignant familiar story young person suspended...,1.0
2,timely director could ever dreamed quietly lyr...,1.0
3,film virtually choke selfconsciousness,0.0
4,film take inside rhythm subject experience watch,1.0
...,...,...
8524,branagh forceful nonshakespeare screen perform...,1.0
8525,movie friday fan critic damned already like so...,0.0
8526,perhaps heaviest joyless movie ever made giant...,0.0
8527,film rival live fine little amusebouche keep a...,1.0


In [15]:
test_data

,review,mood
0,important movie reminder power film move u mak...,1.0
1,ive never seen heard anything quite like film ...,1.0
2,ending leave unfulfilled performance enjoy mem...,1.0
3,surface loversontherun crime flick lot common ...,1.0
4,walk remember shrewd enough activate girlish t...,0.0
...,...,...
2128,bullock good job working natural likability,1.0
2129,result memorable least interesting,1.0
2130,apparently designed reverie memory regret thin...,0.0
2131,movie insecure capacity excite churn one two f...,0.0


In [16]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_data['review'])
test_vectors = vectorizer.transform(test_data['review'])

In [17]:
# from sklearn import svm
# 
# classifier = svm.SVC(kernel='linear')
# classifier.fit(train_vectors , train_data['mood'])
# report = classification_report(test_data['mood'] , pred , output_dict=True)
# print(f"positive {report['1.0']['recall']}")
# print(f"negative {report['0.0']['recall']}")


In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
clf = MultinomialNB()
clf.fit(train_vectors , train_data['mood'])

MultinomialNB()

In [25]:
pred = clf.predict(test_vectors)
pred

array([1., 1., 1., ..., 0., 1., 1.])

In [37]:
report = classification_report(test_data['mood'] , pred , output_dict=True)
print(f"positive {report['1.0']['recall']}")
print(f"negative {report['0.0']['recall']}")
abc=pd.DataFrame(report)
abc

positive 0.7768518518518519
negative 0.7796771130104464


,0.0,1.0,accuracy,macro avg,weighted avg
precision,0.773070,0.783380,0.778247,0.778225,0.778290
recall,0.779677,0.776852,0.778247,0.778264,0.778247
f1-score,0.776359,0.780102,0.778247,0.778231,0.778254
support,1053.000000,1080.000000,0.778247,2133.000000,2133.000000


In [32]:
a  = input('write the review: ')
vector = vectorizer.transform([a]).toarray()
xyz = clf.predict(vector)
print(xyz)

write the review: it was awesome movie
[1.]


In [ ]:
# methods to save the model

# 1. pickle
# 2. joblib

In [ ]:
# # save the model using joblib
# import joblib
# model_file_name = 'netflix_svm_model.pkl'
# vectorizer_filename = 'netflix_vector.pkl'
# joblib.dump(classifier , model_file_name)
# joblib.dump(vectorizer , vectorizer_filename)

In [ ]:
# ls

In [ ]:
# loading a model
# vect = joblib.load('netflix_vector.pkl')
# clf = joblib.load('netflix_svm_model.pkl')

In [ ]:
# take the input from the user
# convert to list
# sent the list to vectoriser so that my string can be converted in a vector form
# sent the vector to the model



# a = 'best'
# b = vect.transform([a])
# clf.predict(b)

In [ ]:
# task create an flask app and put it on github 